In [2]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
from tqdm import tqdm

print("✅ 설치 완료!")

✅ 설치 완료!


In [3]:
import os
from pathlib import Path

In [6]:
# 프로젝트 구조 기준 경로 설정
BASE_DIR = Path.cwd().parent # src/data_preprocesser/에서 한 단계 위
RAW_DIR = BASE_DIR / "data" / "raw" / "LE2i"
PROCESSED_DIR = BASE_DIR / "data" / "processed"

# processed 하위 폴더 생성
(PROCESSED_DIR / "keypoints").mkdir(parents=True, exist_ok=True)
(PROCESSED_DIR / "videos").mkdir(parents=True, exist_ok=True)

In [5]:
# MediaPipe pose 모델 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False,
                    model_complexity=1,
                    enable_segmentation=False,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5)

draw = mp.solutions.drawing_utils

In [6]:
def read_annotation(annotation_path):
    """LE2i 어노테이션 파일에서 낙상 시작/종료 프레임 읽기"""
    with open(annotation_path, 'r') as f:
        lines = f.readlines()

    start_frame = int(lines[0].strip())
    end_frame = int(lines[1].strip())

    if start_frame == 0 and end_frame == 0:
        return None  # 낙상 없음
    return (start_frame, end_frame)

In [12]:
def process_video(video_path, annotation_path, save_keypoints_path, save_video_path=None):
    cap = cv2.VideoCapture(str(video_path))
    keypoints_all = []

    # 어노테이션 읽기
    fall_range = read_annotation(annotation_path)
    
    # 비디오 저장 설정
    out = None
    if save_video_path:
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        fps = cap.get(cv2.CAP_PROP_FPS)
        w, h = int(cap.get(3)), int(cap.get(4))
        out = cv2.VideoWriter(str(save_video_path), fourcc, fps, (w, h))

    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # RGB 변환 (MediaPipe는 RGB 입력)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb_frame)
        
        if results.pose_landmarks:
            frame_keypoints = [[lm.x, lm.y, lm.z, lm.visibility] for lm in results.pose_landmarks.landmark]
        else:
            frame_keypoints = [[0, 0, 0, 0]] * 33  # 사람 감지 안됨

        # 낙상 라벨 추가 (0: 정상, 1: 낙상)
        if fall_range:
            label = 1 if fall_range[0] <= frame_idx <= fall_range[1] else 0
        else:
            label = 0

        keypoints_all.append({"frame": frame_idx, "label": label, "keypoints": frame_keypoints})

        # 영상에 표시
        if out:
            if results.pose_landmarks:
                draw.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            if label == 1:
                cv2.putText(frame, "FALL", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)
            out.write(frame)
        
        frame_idx += 1

    cap.release()
    if out:
        out.release()
    
    # keypoints json 저장
    with open(save_keypoints_path, 'w') as f:
        json.dump(keypoints_all, f)
    
    print(f"Processed {video_path.name}, frames: {frame_idx}")


In [25]:
# 모든 폴더 반복
for scene_folder in RAW_DIR.iterdir():
    if not scene_folder.is_dir():
        continue

    videos_folder = scene_folder / "Videos"
    ann_folder = scene_folder / "Annotation_files"

    for video_file in videos_folder.glob("*.avi"):
        base_name = video_file.stem
        ann_file = ann_folder / f"{base_name}.txt"
        if not ann_file.exists():
            print(f"⚠️ No annotation for {video_file.name}, skipped")
            continue

        save_name = f"{scene_folder.name}_{base_name}"
        keypoints_path = PROCESSED_DIR / "keypoints" / f"{save_name}.json"
        video_out_path = PROCESSED_DIR / "videos" / f"{save_name}.avi"

        if os.path.exists(keypoints_path):
            print(f"⚠️ Skipped (already exists): {keypoints_path}")
            continue
        
        process_video(video_file, ann_file, keypoints_path, video_out_path)

⚠️ Skipped (already exists): C:\Users\un747\capstone1_AI\data\processed\keypoints\Coffee_room_01_video (1).json
⚠️ Skipped (already exists): C:\Users\un747\capstone1_AI\data\processed\keypoints\Coffee_room_01_video (10).json
⚠️ Skipped (already exists): C:\Users\un747\capstone1_AI\data\processed\keypoints\Coffee_room_01_video (11).json
⚠️ Skipped (already exists): C:\Users\un747\capstone1_AI\data\processed\keypoints\Coffee_room_01_video (12).json
⚠️ Skipped (already exists): C:\Users\un747\capstone1_AI\data\processed\keypoints\Coffee_room_01_video (13).json
⚠️ Skipped (already exists): C:\Users\un747\capstone1_AI\data\processed\keypoints\Coffee_room_01_video (14).json
⚠️ Skipped (already exists): C:\Users\un747\capstone1_AI\data\processed\keypoints\Coffee_room_01_video (15).json
⚠️ Skipped (already exists): C:\Users\un747\capstone1_AI\data\processed\keypoints\Coffee_room_01_video (16).json
⚠️ Skipped (already exists): C:\Users\un747\capstone1_AI\data\processed\keypoints\Coffee_room_01_

In [32]:
def read_label_from_txt(txt_path):
    """txt 파일에서 낙상 라벨(0 또는 1) 추출"""
    with open(txt_path, 'r') as f:
        first_line = f.readline().strip()
    label = int(first_line.split()[0])
    return label

def process_caucafall_subject(subject_path, save_dir):
    """Subject 내 모든 scenario 폴더 처리"""
    for scenario_dir in subject_path.iterdir():

        img_files = sorted([p for p in scenario_dir.glob("*.png")])
        if not img_files:
            print(f"⚠️ No image files found in {scenario_dir}")
            return

        keypoints_all = []
        frame_idx = 0
        print(f"Processing {scenario_dir} ...")

        for img_path in tqdm(sorted(scenario_dir.glob("*.png"))):
            txt_path = img_path.with_suffix('.txt')
            if not txt_path.exists():
                continue

            label = read_label_from_txt(txt_path)
            frame = cv2.imread(str(img_path))
            if frame is None:
                continue

            # RGB 변환 후 MediaPipe 처리
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(rgb_frame)

            if results.pose_landmarks:
                frame_keypoints = [
                    [lm.x, lm.y, lm.z, lm.visibility]
                    for lm in results.pose_landmarks.landmark
                ]
            else:
                frame_keypoints = [[0, 0, 0, 0]] * 33  # 감지 실패 시 0으로 채움

            keypoints_all.append({
                "frame": frame_idx,
                "label": label,
                "keypoints": frame_keypoints
            })
            frame_idx += 1

        # 저장 경로 설정 (예: Subject1_scenario1.json)
        save_path = save_dir / f"{subject_path.name}_{scenario_dir.name}.json"
        with open(save_path, 'w') as f:
            json.dump(keypoints_all, f)
        print(f"Saved: {save_path}")

def process_caucafall_dataset(raw_dir, save_dir):
    """CAUCAFall 전체 데이터셋 처리"""
    for subject_dir in sorted(raw_dir.iterdir()):
        if subject_dir.is_dir():
            process_caucafall_subject(subject_dir, save_dir)

In [36]:
# MediaPipe pose 모델 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False,
                    model_complexity=1,
                    enable_segmentation=False,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5)

draw = mp.solutions.drawing_utils

RAW_DIR = BASE_DIR / "data" / "raw" / "CAUCAFall"
process_caucafall_dataset(RAW_DIR, PROCESSED_DIR / "keypoints")

pose.close()

Processing C:\Users\un747\capstone1_AI\data\raw\CAUCAFall\Subject.1\Fall backwards ...


C:\Users\un747\anaconda3\envs\fall\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
100%|████████████████████████████████████████████████████████████████████████████████| 126/126 [00:16<00:00,  7.84it/s]


Saved: C:\Users\un747\capstone1_AI\data\processed\keypoints\Subject.1_Fall backwards.json
Processing C:\Users\un747\capstone1_AI\data\raw\CAUCAFall\Subject.1\Fall forward ...


100%|████████████████████████████████████████████████████████████████████████████████| 191/191 [00:30<00:00,  6.30it/s]


Saved: C:\Users\un747\capstone1_AI\data\processed\keypoints\Subject.1_Fall forward.json
Processing C:\Users\un747\capstone1_AI\data\raw\CAUCAFall\Subject.1\Fall left ...


...

Saved: C:\Users\un747\capstone1_AI\data\processed\keypoints\Subject.9_Pick up object.json
Processing C:\Users\un747\capstone1_AI\data\raw\CAUCAFall\Subject.9\Sit down ...


100%|████████████████████████████████████████████████████████████████████████████████| 239/239 [00:36<00:00,  6.54it/s]


Saved: C:\Users\un747\capstone1_AI\data\processed\keypoints\Subject.9_Sit down.json
Processing C:\Users\un747\capstone1_AI\data\raw\CAUCAFall\Subject.9\Walk ...


100%|████████████████████████████████████████████████████████████████████████████████| 235/235 [00:35<00:00,  6.67it/s]


Saved: C:\Users\un747\capstone1_AI\data\processed\keypoints\Subject.9_Walk.json


In [42]:
import json
from pathlib import Path

def unify_fall_labels(processed_dir, pre_fall_frames=20, post_fall_frames=10):
    """
    processed_dir 내부의 모든 JSON 파일을 순회하면서
    - 낙상(label=1)이 있는 경우, 낙상 시작 전 pre_fall_frames, 이후 post_fall_frames을 포함하도록 수정
    - 낙상(label=1)이 없는 파일은 그대로 둠
    """
    processed_dir = Path(processed_dir)
    json_files = list(processed_dir.rglob("*.json"))

    modified_count = 0

    for json_path in json_files:
        with open(json_path, 'r') as f:
            data = json.load(f)

        labels = [frame["label"] for frame in data]
        
        if sum(labels) == 0:
            # 낙상 없는 파일은 건드리지 않음
            continue
        
        # 낙상이 시작되는 첫 프레임과 마지막 프레임 찾기
        fall_indices = [i for i, l in enumerate(labels) if l == 1]
        fall_start = fall_indices[0]
        fall_end = fall_indices[-1]

        # 통일된 범위: 시작 이전 pre_fall_frames, 끝 이후 post_fall_frames
        start = max(fall_start - pre_fall_frames, 0)
        end = min(fall_start + post_fall_frames, len(labels) - 1)

        # 라벨 수정
        for i in range(start, end + 1):
            data[i]["label"] = 1

        for i in range(end + 1, len(labels)):
            data[i]["label"] = 0

        # 덮어쓰기 저장
        with open(json_path, 'w') as f:
            json.dump(data, f, indent=2)

        modified_count += 1
        print(f"✅ Updated: {json_path.name} (frames {start}~{end} marked as fall)")

    print(f"\n총 {modified_count}개의 낙상 파일 수정 완료 ✅")

In [46]:
# 사용 예시

unify_fall_labels(BASE_DIR / "data" / "processed" / "keypoints" / "CAUCAFall_js", pre_fall_frames=20, post_fall_frames=10)

✅ Updated: Subject.10_Fall backwards.json (frames 58~88 marked as fall)
✅ Updated: Subject.10_Fall forward.json (frames 33~63 marked as fall)
✅ Updated: Subject.10_Fall left.json (frames 30~60 marked as fall)
✅ Updated: Subject.10_Fall right.json (frames 54~84 marked as fall)
✅ Updated: Subject.10_Fall sitting.json (frames 56~86 marked as fall)
✅ Updated: Subject.1_Fall backwards.json (frames 38~68 marked as fall)
✅ Updated: Subject.1_Fall forward.json (frames 111~141 marked as fall)
✅ Updated: Subject.1_Fall left.json (frames 24~54 marked as fall)
✅ Updated: Subject.1_Fall right.json (frames 36~66 marked as fall)
✅ Updated: Subject.1_Fall sitting.json (frames 78~108 marked as fall)
✅ Updated: Subject.2_Fall backwards.json (frames 43~73 marked as fall)
✅ Updated: Subject.2_Fall forward.json (frames 4~34 marked as fall)
✅ Updated: Subject.2_Fall left.json (frames 12~42 marked as fall)
✅ Updated: Subject.2_Fall right.json (frames 13~43 marked as fall)
✅ Updated: Subject.2_Fall sitting.js

In [20]:
import json
import numpy as np
from pathlib import Path

# 경로 설정
BASE_DIR = Path.cwd().parent   # src의 상위 폴더
DATA_DIR = BASE_DIR / "data" / "processed" / "keypoints"  # JSON 폴더
SAVE_DIR = BASE_DIR / "data" / "processed" / "npz"        # npz 저장 폴더

SAVE_DIR.mkdir(parents=True, exist_ok=True)

FALL_WINDOW_BEFORE = 40
FALL_WINDOW_AFTER = 40
NORMAL_WINDOW = 80
NORMAL_STRIDE = 40
# =================

FALL_LEN = FALL_WINDOW_BEFORE + FALL_WINDOW_AFTER
NORMAL_LEN = NORMAL_WINDOW
# ======================

X, Y = [], []


def pad_or_trim(sequence, target_len):
    """시퀀스를 target_len에 맞게 패딩 또는 자르기"""
    seq_len = len(sequence)
    feature_dim = sequence.shape[1]

    if seq_len == target_len:
        return sequence
    elif seq_len > target_len:
        return sequence[:target_len]
    else:
        pad_len = target_len - seq_len
        pad = np.zeros((pad_len, feature_dim), dtype=np.float32)
        return np.vstack([sequence, pad])


def extract_clip(frames, start_idx, end_idx):
    """frames[start_idx:end_idx]의 관절 keypoints 추출"""
    clip = []
    for f in frames[start_idx:end_idx]:
        key = f["keypoints"]
        flat = [v for kp in key for v in kp]  # 33 joints × 4 = 132D
        clip.append(flat)
    return np.array(clip, dtype=np.float32)


def process_json(json_path):
    """하나의 json 파일에서 낙상/비낙상 구간 클립 추출"""
    with open(json_path, "r") as f:
        data = json.load(f)

    labels = [frame["label"] for frame in data]
    has_fall = 1 in labels

    # ---------- 낙상 ----------
    if has_fall:
        fall_start = labels.index(1)
        start_idx = max(0, fall_start - FALL_WINDOW_BEFORE)
        end_idx = min(len(data), fall_start + FALL_WINDOW_AFTER)
        clip = extract_clip(data, start_idx, end_idx)
        clip = pad_or_trim(clip, FALL_LEN)
        X.append(clip)
        Y.append(1)

    # ---------- 비낙상 ----------
    else:
        total_frames = len(data)
        for start in range(0, total_frames - NORMAL_WINDOW + 1, NORMAL_STRIDE):
            end = start + NORMAL_WINDOW
            clip = extract_clip(data, start, end)
            clip = pad_or_trim(clip, NORMAL_LEN)
            X.append(clip)
            Y.append(0)


# ==============================
# 모든 json 처리
# ==============================
for json_file in sorted(DATA_DIR.glob("*.json")):
    process_json(json_file)

# numpy 배열로 변환 (이제 shape 일정)
X = np.array(X, dtype=np.float32)
Y = np.array(Y, dtype=np.int32)

np.savez_compressed(SAVE_DIR / "fall_dataset.npz", X=X, Y=Y)

print(f"✅ npz 파일 저장 완료: {SAVE_DIR / 'fall_dataset.npz'}")
print(f"총 샘플 수: {len(X)} (낙상: {sum(Y)}, 비낙상: {len(Y)-sum(Y)})")
print(f"X shape: {X.shape} | Y shape: {Y.shape}")

✅ npz 파일 저장 완료: C:\Users\un747\capstone1_AI\data\processed\npz\fall_dataset.npz
총 샘플 수: 603 (낙상: 146, 비낙상: 457)
X shape: (603, 80, 132) | Y shape: (603,)
